In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
stock_dict = {
    'MEG': [],
    'JGS': [],
    'BDO': [],
    'FGEN': [],
    'ICT': [],
    'ALI': [],
    'SMC': [],
    'TEL': [],
    'GLO': [],
    'BLOOM': [],
    'RLC': [],
    'MER': [],
    'AC': [],
    'PGOLD': [],
    'LTG': [],
    'MPI': [],
    'AP': [],
    'RRHI': [],
    'URC': [],
    'PSEI': [],
}

In [3]:
for index, key in enumerate(stock_dict):
    # Loading of datasets to the dictionary of stocks
    # We will only use the closing prices, hence we set usecols to index:4
    data = np.genfromtxt(f'data/{key}.csv', delimiter=',', skip_header=1, usecols=4)
    stock_dict[key] = data

print(stock_dict)

{'MEG': array([2.3408, 2.3145, 2.2882, ..., 2.37  , 2.31  , 2.34  ]), 'JGS': array([ 7.6149,  7.695 ,  7.695 , ..., 54.    , 53.6   , 53.9   ]), 'BDO': array([ 17.8868,  17.8868,  17.8868, ..., 126.    , 125.    , 125.5   ]), 'FGEN': array([19.5961, 19.8114, 19.8114, ..., 18.1   , 18.18  , 18.02  ]), 'ICT': array([  4.9185,   4.9185,   4.6815, ..., 212.    , 211.4   , 214.    ]), 'ALI': array([ 6.9136,  6.9136,  9.3827, ..., 29.2   , 29.1   , 29.25  ]), 'SMC': array([ 45.8774,  45.8774,  43.9855, ..., 112.7   , 110.    , 107.5   ]), 'TEL': array([ 827.5,  827.5,  700. , ..., 1395. , 1380. , 1380. ]), 'GLO': array([ 749.8907,  759.6295,  740.1518, ..., 2090.    , 2080.    ,
       2016.    ]), 'BLOOM': array([0.72, 0.7 , 0.9 , ..., 9.4 , 9.44, 9.45]), 'RLC': array([ 2.9573,  2.9957,  3.3029, ..., 16.42  , 16.3   , 16.14  ]), 'MER': array([ 95.9033,  95.1712,  97.3675, ..., 294.    , 283.    , 315.    ]), 'AC': array([198.2492, 195.6745, 205.9732, ..., 714.    , 700.    , 700.    ]), 'PG

In [4]:
stock_symbols = [keys for _,keys in enumerate(stock_dict)]
print(stock_symbols)

['MEG', 'JGS', 'BDO', 'FGEN', 'ICT', 'ALI', 'SMC', 'TEL', 'GLO', 'BLOOM', 'RLC', 'MER', 'AC', 'PGOLD', 'LTG', 'MPI', 'AP', 'RRHI', 'URC', 'PSEI']


In [6]:
# Define the function for ALMA
def alma(data, window_size:int, sigma:float, offset:float):
    m = offset * (window_size - 1)
    s = window_size / sigma
    w = np.exp(-(np.arange(window_size) - m)**2 / (2 * s**2))
    w /= w.sum()
    return np.convolve(data, w, mode='valid')

In [7]:
# Backtesting
def back_testing(slow_alma_data, fast_alma_data, data):
    # Initialize the variables
    position = 0
    entry_price = 0
    exit_price = 0
    profit = 0

    # Initialize the lists
    entry_price_list = []
    exit_price_list = []

    # Initialize the counters
    position_counter = 0
    profit_counter = 0

    # Initialize the flags
    entry_flag = False
    exit_flag = False

    # Loop through the data (slow_alma_meg, fast_alma_meg, data_meg)
    for i, j, k in zip(slow_alma_data, fast_alma_data, data):
        # If we are flat
        if position == 0:
            # If the fast ALMA crosses above the slow ALMA
            if j > i:
                # Set the entry price
                entry_price = k
                # Set the entry flag to True
                entry_flag = True
                # Set the exit flag to False
                exit_flag = False
                # Set the position to 1
                position = 1
                # Increment the position counter by 1
                position_counter += 1
        # If we are long
        elif position == 1:
            # If the fast ALMA crosses below the slow ALMA
            if j < i:
                # Set the exit price
                exit_price = k
                # Set the exit flag to True
                exit_flag = True
                # Set the entry flag to False
                entry_flag = False
                # Set the position to 0
                position = 0
                # Calculate the profit
                profit = exit_price - entry_price
                # Increment the profit counter by the profit
                profit_counter += profit
                # Append the entry price to the entry price list
                entry_price_list.append(entry_price)
                # Append the exit price to the exit price list
                exit_price_list.append(exit_price)

    return profit_counter


In [8]:
# backtesting2
# buy: mean fast alma in 10 days > mean slow alma in 10 days
# sell: mean fast alma in 10 days < mean slow alma in 10 days
# hold: no change in previous position
def back_testing2(slow_alma_data, fast_alma_data, data):
    current_position = 0 # 0 - None, 1 - Buy, -1 - Sell
    profit = 0
    
    # Divide the data into 10 days
    slow_alma_data = np.array_split(slow_alma_data, len(slow_alma_data)/10)
    fast_alma_data = np.array_split(fast_alma_data, len(fast_alma_data)/10)
    data = np.array_split(data, len(data)/10)

    for i, j, k in zip(slow_alma_data, fast_alma_data, data):
        if current_position == 0:
            if np.mean(j) > np.mean(i):
                current_position = 1
            elif np.mean(j) < np.mean(i):
                current_position = -1
        elif current_position == 1:
            if np.mean(j) < np.mean(i):
                current_position = 0
                profit += k[-1] - k[0]
        elif current_position == -1:
            if np.mean(j) > np.mean(i):
                current_position = 0
                profit += k[0] - k[-1]
    
    return profit


In [10]:
# Create a dataframe for the results
results_signalbase = pd.DataFrame(columns=['stock', 'expectedProfit'])

# Loop through the stocks
for index, key in enumerate(stock_dict):
    # Get the data
    data = stock_dict[key]
    slow_alma_baseline = alma(data, 10, 17, 0.95)
    fast_alma_baseline = alma(data, 6, 13, 0.85)
    profit = back_testing(slow_alma_baseline, fast_alma_baseline, data)

    # Append the results to the dataframe (using pd.concat)
    results_signalbase = pd.concat([results_signalbase, pd.DataFrame([[key, profit]], columns=['stock', 'expectedProfit'])])
    # remove the index
    results_signalbase = results_signalbase.reset_index(drop=True)
results_signalbase

,stock,expectedProfit
0,MEG,149.2233
1,JGS,1569.8650
2,BDO,2541.9970
3,FGEN,581.0804
4,ICT,2815.6103
5,ALI,1072.1418
6,SMC,2557.0770
7,TEL,72070.5000
8,GLO,60538.0035
9,BLOOM,495.4600


In [11]:
# Create a dataframe for the results
results_meanbase = pd.DataFrame(columns=['stock', 'expectedProfit'])

# Loop through the stocks
for index, key in enumerate(stock_dict):
    # Get the data
    data = stock_dict[key]
    slow_alma_baseline = alma(data, 18, 12, 0.85)
    fast_alma_baseline = alma(data, 9, 1, 0.85)
    profit = back_testing2(slow_alma_baseline, fast_alma_baseline, data)

    # Append the results to the dataframe (using pd.concat)
    results_meanbase = pd.concat([results_meanbase, pd.DataFrame([[key, profit]], columns=['stock', 'expectedProfit'])])
    # remove the index
    results_meanbase = results_meanbase.reset_index(drop=True)
results_meanbase

,stock,expectedProfit
0,MEG,30.6186
1,JGS,319.0707
2,BDO,626.3279
3,FGEN,171.9919
4,ICT,546.0577
5,ALI,-121.7964
6,SMC,-385.6427
7,TEL,17080.0000
8,GLO,-8101.9688
9,BLOOM,155.5300
